<a href="https://colab.research.google.com/github/Bprs68/Rossmann-Sales-Prediction-/blob/main/Notebook_2_Baseline_Model_Building_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import math

import warnings    
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/AlmaBetter/Capstone Projects/Capstone Project - Supervised Machine Learning- Regression/cleaned_data/'

In [ ]:
train_df = pd.read_csv(file_path+'train.csv')

In [ ]:
test_df = pd.read_csv(file_path+'test.csv')

In [ ]:
#setting date and store as index
train_df.set_index(['Date','Store'], inplace=True)
train_df.sort_values(by=['Date','Store'], inplace=True)

test_df.set_index(['Date','Store'], inplace=True)
test_df.sort_values(by=['Date','Store'], inplace=True)

In [ ]:
train_df.head(1)

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,WeekOfYear,DayOfYear,CompetitionOpen,Promo2Open,Promo2running
Date,Store,,,,,,,,,,,,,,,,,
2013-01-01,85,2,8.34759,619,0,1,1,b,a,1870.0,0,2013,1,1,1,15.0,0.0,0


In [ ]:
train_df.columns

Index(['DayOfWeek', 'Sales', 'Customers', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'Promo2', 'Year', 'Month', 'WeekOfYear', 'DayOfYear', 'CompetitionOpen',
       'Promo2Open', 'Promo2running'],
      dtype='object')

In [ ]:
test_df.head(1)

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,WeekOfYear,DayOfYear,CompetitionOpen,Promo2Open,Promo2running
Date,Store,,,,,,,,,,,,,,,,,
2015-06-15,1,1,8.615771,586,1,0,0,c,a,1270.0,0,2015,6,25,166,81.0,0.0,0


## Test-Train-Split

In [ ]:
# dividing train df into train_x and train_y 
X_train = train_df.drop('Sales', axis=1)
y_train = train_df[['Sales']]

#dividing test df into test_x and test_y
X_test = test_df.drop("Sales", axis=1)
y_test = test_df[['Sales']]

One Hot Encoding of categorical columns to conver them into numerical columns

In [ ]:
# making a list of categorical columns
Categorical_columns = ['DayOfWeek','StoreType','Assortment']

In [ ]:
# assigning one hot encoder
enc = OneHotEncoder(sparse=False)

In [ ]:
enc.fit(X_train[Categorical_columns])

OneHotEncoder(sparse=False)

In [ ]:
# getting a new list of encoded columns from Categorical_columns in train data
encoded_cols = enc.get_feature_names(Categorical_columns).tolist()

In [ ]:
encoded_cols

['DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c']

In [ ]:
#performing transformation on training data and creating new features for encoded columns
X_train[encoded_cols] = enc.transform(X_train[Categorical_columns])

In [ ]:
#dropping old categorical columns from training data
X_train.drop(Categorical_columns, axis=1, inplace=True)

In [ ]:
#performing transformation on testing data and creating new features for encoded columns
X_test[encoded_cols] = enc.transform(X_test[Categorical_columns])

In [ ]:
#dropping old categorical columns from testing data
X_test.drop(Categorical_columns, axis=1, inplace=True)

In [ ]:
X_train.columns

Index(['Customers', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'CompetitionDistance', 'Promo2', 'Year', 'Month', 'WeekOfYear',
       'DayOfYear', 'CompetitionOpen', 'Promo2Open', 'Promo2running',
       'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
       'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'StoreType_a',
       'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_a',
       'Assortment_b', 'Assortment_c'],
      dtype='object')

### **Time for transformations in our data**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[list(X_train.columns)] = scaler.fit_transform(X_train[list(X_train.columns)])
X_test[list(X_test.columns)] = scaler.fit_transform(X_test[list(X_test.columns)])

scaler = StandardScaler()
y_train[list(y_train.columns)] = scaler.fit_transform(y_train[list(y_train.columns)])
y_test[list(y_test.columns)] = scaler.transform(y_test[list(y_train.columns)])

## **Model Selection**

Going through the assumptions of linear models, we can confidently conclude that we can go for them. Given Our Data has a lot of genuine multicollinearity and also some columns have way too many outliers than others. 

Linear models like Linear regression and Logistic ones can't be used for our purpose, so we will move ahead with decision trees and randomforests, but, before that just to check.

In [ ]:
lrreg = LinearRegression()

In [ ]:
lrreg.fit(X_train,y_train)

LinearRegression()

In [ ]:
y_train_pred = lrreg.predict(X_train)

In [ ]:
y_test_pred = lrreg.predict(X_test)

In [ ]:
# importing evaluation matrices
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



In [ ]:
print(f'r2_score for training data is {r2_score(y_train_pred,y_train)}')
print(f'r2_score for training data is {r2_score(y_test_pred,y_test)}')

r2_score for training data is 0.6675621318983829
r2_score for training data is -3.831468475823385e-11


As expected high bias and high variance.

### **Model 1(Baseline): DecisionTree**

In [ ]:
# import the regressor
from sklearn.tree import DecisionTreeRegressor 

In [ ]:
# assigning a variable
dtree = DecisionTreeRegressor(random_state=68)

In [ ]:
dtree.fit(X_train,y_train)

DecisionTreeRegressor(random_state=68)

In [ ]:
# predicting Y_train
y_pred_train = dtree.predict(X_train)

In [ ]:
# predicting Y_train
y_pred_test = dtree.predict(X_test)

In [ ]:
# printing evaluation matrices for our model
print(f'r2_score for training data is {r2_score(y_pred_train,y_train)}')
print(f'r2_score for testing data is {r2_score(y_pred_test,y_test)}')

print(f'Adjusted r2_score for training data is {round(1 - (1-r2_score(y_train, y_pred_train)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)}')
print(f'Adjusted r2_score for testing data is {round(1 - (1-r2_score(y_test, y_pred_test)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)}')

print(f'Mean Absolute Error for training data is {mean_absolute_error(y_pred_train,y_train)}')
print(f'Mean Absolute Error for testing data is {mean_absolute_error(y_pred_test,y_test)}')

print(f'Mean Squared Error for training data is {mean_squared_error(y_pred_train,y_train)}')
print(f'Mean Squared Error for testing data is {mean_squared_error(y_pred_test,y_test)}')

print(f'Root Mean Squared Error for training data is {math.sqrt(mean_squared_error(y_pred_train,y_train))}')
print(f'Root Mean Squared Error for testing data is {math.sqrt(mean_squared_error(y_pred_test,y_test))}')

r2_score for training data is 0.9999987542867533
r2_score for testing data is 0.9132782268502603
Adjusted r2_score for training data is 0.999999
Adjusted r2_score for testing data is 0.911298
Mean Absolute Error for training data is 4.189885590080446e-06
Mean Absolute Error for testing data is 0.20540436100243337
Mean Squared Error for training data is 1.2457116948123794e-06
Mean Squared Error for testing data is 0.08350558487359472
Root Mean Squared Error for training data is 0.0011161145527285178
Root Mean Squared Error for testing data is 0.2889733290004369


As it is famously assumed, Decision trees are indeed prone to overfitting, with r2_score of 1 on traing data, it has completely overfitted the data, while for test data, it's accuracy is around 93%.

To ease this problem of overfitting we will use Random Forest to improve our model accuracy.

### **Model 2: Random Forest**

A single tree was not able to lift much weight, so now we will use entire forest.

In [ ]:
# importing random forest from ScikitLearn
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=68)

In [ ]:
rf.fit(X_train,y_train)

RandomForestRegressor(random_state=68)

In [ ]:
y_pred_train_rf = rf.predict(X_train)

In [ ]:
y_pred_test_rf = rf.predict(X_test)

In [ ]:
# printing evaluation matrices for our model
print(f'r2_score for training data is {r2_score(y_pred_train_rf,y_train)}')
print(f'r2_score for testing data is {r2_score(y_pred_test_rf,y_test)}')

print(f'Adjusted r2_score for training data is {round(1 - (1-r2_score(y_train, y_pred_train_rf)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)}')
print(f'Adjusted r2_score for testing data is {round(1 - (1-r2_score(y_test, y_pred_test_rf)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)}')

print(f'Mean Absolute Error for training data is {mean_absolute_error(y_pred_train_rf,y_train)}')
print(f'Mean Absolute Error for testing data is {mean_absolute_error(y_pred_test_rf,y_test)}')

print(f'Mean Squared Error for training data is {mean_squared_error(y_pred_train_rf,y_train)}')
print(f'Mean Squared Error for testing data is {mean_squared_error(y_pred_test_rf,y_test)}')


print(f'Root Mean Squared Error for training data is {math.sqrt(mean_squared_error(y_pred_train_rf,y_train))}')
print(f'Root Mean Squared Error for testing data is {math.sqrt(mean_squared_error(y_pred_test_rf,y_test))}')

r2_score for training data is 0.9964356349429113
r2_score for testing data is 0.9508591528079314
Adjusted r2_score for training data is 0.9965
Adjusted r2_score for testing data is 0.952252
Mean Absolute Error for training data is 0.04334227746186602
Mean Absolute Error for testing data is 0.1538746851395498
Mean Squared Error for training data is 0.003500113007311803
Mean Squared Error for testing data is 0.0449506975157633
Root Mean Squared Error for training data is 0.059161752909390734
Root Mean Squared Error for testing data is 0.2120157954393099


Our r2_score for training data is almost 99% while for test data it is 95%. It seems all good, but let's try finding best parameters to see if we can further improve our score.

### **Model 3: LGBM**

In [ ]:
# importing lgbm
from lightgbm import LGBMRegressor

In [ ]:
# assigning variable
lgbm_reg = LGBMRegressor(random_state=68)

In [ ]:
# fitting on training data
lgbm_reg.fit(X_train,y_train)

LGBMRegressor(random_state=68)

In [ ]:
# predicting on y_train
y_train_pred_lgbm = lgbm_reg.predict(X_train)

In [ ]:
# predicting on y_test
y_test_pred_lgbm = lgbm_reg.predict(X_test)

In [ ]:
# printing accuracy
# printing evaluation matrices for our model
print(f'r2_score for training data is {r2_score(y_train_pred_lgbm,y_train)}')
print(f'r2_score for testing data is {r2_score(y_test_pred_lgbm,y_test)}')

print(f'Adjusted r2_score for training data is {round(1 - (1-r2_score(y_train, y_train_pred_lgbm)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)}')
print(f'Adjusted r2_score for testing data is {round(1 - (1-r2_score(y_test, y_test_pred_lgbm)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)}')

print(f'Mean Absolute Error for training data is {mean_absolute_error(y_train_pred_lgbm,y_train)}')
print(f'Mean Absolute Error for testing data is {mean_absolute_error(y_test_pred_lgbm,y_test)}')

print(f'Mean Squared Error for training data is {mean_squared_error(y_train_pred_lgbm,y_train)}')
print(f'Mean Squared Error for testing data is {mean_squared_error(y_test_pred_lgbm,y_test)}')

print(f'Root Mean Squared Error for training data is {math.sqrt(mean_squared_error(y_train_pred_lgbm,y_train))}')
print(f'Root Mean Squared Error for testing data is {math.sqrt(mean_squared_error(y_test_pred_lgbm,y_test))}')


r2_score for training data is 0.9156963683049166
r2_score for testing data is 0.8854440634180716
Adjusted r2_score for training data is 0.925448
Adjusted r2_score for testing data is 0.896417
Mean Absolute Error for training data is 0.2148672858875459
Mean Absolute Error for testing data is 0.24558678172808118
Mean Squared Error for training data is 0.07454970345199258
Mean Squared Error for testing data is 0.09751441413804593
Root Mean Squared Error for training data is 0.2730379157772645
Root Mean Squared Error for testing data is 0.3122729801600611


We have made predictions using default parameters of Light Gradient Boosting model, simply to get and idea of how it might perfrm on our training and test data.

In [ ]:
# creating a comparison df
{'Model_name':['Decision Tree', 'Random Forest Regressor', 'LightGradient Boost Regressor'], 'Variable':['dtree', 'rf', 'lgbm_reg'], 'Train_r2':[r2_score(y_pred_train,y_train), r2_score(y_pred_train_rf,y_train), r2_score(y_train_pred_lgbm,y_train)]}

{'Model_name': ['Decision Tree',
  'Random Forest Regressor',
  'LightGradient Boost Regressor'],
 'Variable': ['dtree', 'rf', 'lgbm_reg'],
 'Train_r2': [0.9999987542867533, 0.9964356349429113, 0.9156963683049166]}

In [ ]:
comparison_df = pd.DataFrame({'Model_name':['Decision Tree', 'Random Forest Regressor', 'LightGradient Boost Regressor'], 'Variable':['dtree', 'rf', 'lgbm_reg'], 'Train_r2':[r2_score(y_pred_train,y_train), r2_score(y_pred_train_rf,y_train), r2_score(y_train_pred_lgbm,y_train)]})

In [ ]:
comparison_df = [r2_score(y_pred_test,y_test), r2_score(y_pred_test_rf,y_test), r2_score(y_test_pred_lgbm,y_test)]
comparison_df

,Model_name,Variable,Train_r2,Test_r2
0,Decision Tree,dtree,0.999999,0.913278
1,Random Forest Regressor,rf,0.996436,0.950859
2,LightGradient Boost Regressor,lgbm_reg,0.915696,0.885444


In [ ]:
comparison_df['Train_adjusted_r2'] = [round(1 - (1-r2_score(y_train, y_pred_train)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6),
                                      round(1 - (1-r2_score(y_train, y_pred_train_rf)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6),
                                      round(1 - (1-r2_score(y_train, y_train_pred_lgbm)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)]

In [ ]:
comparison_df['Test_adjusted_r2'] = [round(1 - (1-r2_score(y_test, y_pred_test)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6),
                                      round(1 - (1-r2_score(y_test, y_pred_test_rf)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6),
                                      round(1 - (1-r2_score(y_test, y_test_pred_lgbm)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)]

In [ ]:
comparison_df['Train_MAE'] = [mean_absolute_error(y_pred_train,y_train),
                              mean_absolute_error(y_pred_train_rf,y_train),
                              mean_absolute_error(y_train_pred_lgbm,y_train)]

In [ ]:
comparison_df['Test_MAE'] = [mean_absolute_error(y_pred_test,y_test),
                             mean_absolute_error(y_pred_test_rf,y_test),
                             mean_absolute_error(y_test_pred_lgbm,y_test)]

In [ ]:
comparison_df['Train_MSE'] = [mean_squared_error(y_pred_train,y_train),
                              mean_squared_error(y_pred_train_rf,y_train),
                              mean_squared_error(y_train_pred_lgbm,y_train)]

In [ ]:
comparison_df['Test_MSE'] = [mean_squared_error(y_pred_test,y_test),
                             mean_squared_error(y_pred_test_rf,y_test),
                             mean_squared_error(y_test_pred_lgbm,y_test)]

In [ ]:
comparison_df['Train_RMSE'] = [math.sqrt(mean_squared_error(y_pred_train,y_train)),
                               math.sqrt(mean_squared_error(y_pred_train_rf,y_train)),
                               math.sqrt(mean_squared_error(y_train_pred_lgbm,y_train))]

In [ ]:
comparison_df['Test_RMSE'] = [math.sqrt(mean_squared_error(y_pred_test,y_test)),
                              math.sqrt(mean_squared_error(y_pred_test_rf,y_test)),
                              math.sqrt(mean_squared_error(y_test_pred_lgbm,y_test))]

In [ ]:
comparison_df

,Model_name,Variable,Train_r2,Test_r2,Train_adjusted_r2,Test_adjusted_r2,Train_MAE,Test_MAE,Train_MSE,Test_MSE,Train_RMSE,Test_RMSE
0,Decision Tree,dtree,0.999999,0.913278,0.999999,0.911298,0.000004,0.205404,0.000001,0.083506,0.001116,0.288973
1,Random Forest Regressor,rf,0.996436,0.950859,0.996500,0.952252,0.043342,0.153875,0.003500,0.044951,0.059162,0.212016
2,LightGradient Boost Regressor,lgbm_reg,0.915696,0.885444,0.925448,0.896417,0.214867,0.245587,0.074550,0.097514,0.273038,0.312273


In [ ]:
c_df = comparison_df.to_csv(file_path+'comparison_df.csv')